**MySQL子查询**

MySQL子查询是嵌套在另一个查询(如SELECT，INSERT，UPDATE或DELETE)中的查询。 另外，MySQL子查询可以嵌套在另一个子查询中。
MySQL子查询称为内部查询，而包含子查询的查询称为外部查询。 子查询可以在使用表达式的任何地方使用，并且必须在括号中关闭

查询返回在位于美国(USA)的办公室工作的员工。    
**SELECT CONCAT(firstname,' ',lastname) AS fullname FROM employees WHERE officeCode IN (SELECT officeCode FROM offices WHERE country = 'USA')**

**MySQL子查询在WHERE子句中使用**

使用payments表来做示例。表结构如下：![payment-structure.png](./images/payments-strcture.png)

**MySQL子查询与比较运算符**

可以使用比较运算符，例如=，>，<等将子查询返回的单个值与WHERE子句中的表达式进行比较。以下查询返回最大付款额的客户。    
**SELECT customerNumber,checkNumber,amount FROM payments WHERE amount = (SELECT MAX(amount) FROM payments);**

可以使用子查询找到其付款大于平均付款的客户。 首先，使用子查询来计算使用AVG聚合函数的平均付款。 然后，在外部查询中，查询大于子查询返回的平均付款的付款。    
**SELECT customerNumber,amount FROM payments WHERE amount > (SELECT AVG(amount) FROM payments);**

**具有IN和NOT IN 运算符的MySQL子查询**

如果子查询返回多个值，则可以在WHERE子句中使用IN或NOT IN运算符等其他运算符。
查看以下客户和订单表的ER结构图![ER-customers-orders](./images/ER-customers-orders.png)

可以使用带有NOT IN运算符的子查询来查找没有下过任何订单的客户    
**SELECT customerNumber,customerName FROM customers WHERE customerNumber NOT IN (SELECT customerNumber FROM orders);**

**FROM 子句中的MySQL子查询**

在FROM子句中使用子查询时，从子查询返回的结果集将用作临时表。 该表称为派生表或物化子查询。以下子查询将查找订单表中的最大，最小和平均数：   
**SELECT MAX(items),MIN(items),FLOOR(AVG(items)) FROM (SELECT orderNumber,COUNT(orderNumber) AS items FROM orderdetails GROUP BY orderNumber) AS lineitems;**

**MySQL相关子查询**

与独立子查询不同，相关子查询是使用外部查询中的数据的子查询。 换句话说，相关的子查询取决于外部查询。 对外部查询中的每一行对相关子查询进行一次评估。    
我们查询选择购买价格高于每个产品线中的产品的平均购买价格的产品。 
**SELECT productName,buyPrice FROM products AS p1 WHERE buyPrice > (SELECT AVG(buyPrice) FROM products WHERE productLine = p1.productLine);**   
对于变化的每一行产品线，每个产品线都会执行内部查询。 因此，平均购买价格也会改变。 外部查询仅筛选购买价格大于子查询中每个产品线的平均购买价格的产品。

**MySQL子查询与EXISTS和NOT EXISTS**

当子查询与EXISTS或NOT EXISTS运算符一起使用时，子查询返回一个布尔值为TRUE或FALSE的值。以下查询说明了与EXISTS运算符一起使用的子查询：   
**SELECT * FROM table_name WHERE EXISTS( subquery );**   
如果子查询(subquery)有返回任何行，则EXISTS子查询返回TRUE，否则返回FALSE。

orders和orderdetails作为示例，表的ER图如下：![ER-orders-orderdetails](./images/ER-orders-orderdetails.png)

查询选择总额大于60000的销售订单。   
**SELECT orderNumber,SUM(quantityOrdered * priceEach) AS total FROM orderdetails INNER JOIN orders USING(orderNumber) GROUP BY orderNumber HAVING total > 60000;**    

注意这里有个坑，WHERE是不能使用别名进行查询的。SQL语句的语法顺序：   
FROM -> WHERE -> GROUP BY -> HAVING -> SELECT -> DISTINCT -> UNION -> ORDER BY，因此一般不能在having condition中使用select list中的alias。**而HAVING在MySQL语句中可以，是因为做了拓展。**   

1、允许在select list、having condition和order by list中使用没有出现在group by list中的字段。

此时mysql会随机选择没有出现在group by list中的字段的值。效果和使用ANY_VALUE()是相同的。

2、允许在having condition中使用select list中的alias

**SELECT customerNumber,customerName FROM customers WHERE EXISTS(SELECT orderNumber,SUM(priceEach \* quantityOrdered) FROM orderdetails INNER JOIN orders USING(orderNumber) WHERE customerNumber = customers.customerNumber GROUP BY orderNumber HAVING SUM(priceEach \* quantityOrdered) > 60000);**

分析语句的执行顺序：
- 外查询FROM
- 子查询FROM 
- 子查询USING
- 子查询WHERE
- 子查询INNER JOIN 
- 子查询GROUP BY
- 子查询HAVING
- 子查询SELECT
- 外查询WHERE
- 外查询SELECT

**MySQL派生表**

派生表是从SELECT语句返回的虚拟表。派生表类似于临时表，但是在SELECT语句中使用派生表比临时表简单得多，因为它不需要创建临时表的步骤。

术语:派生表和子查询通常可互换使用。当SELECT语句的FROM子句中使用独立子查询时，我们将其称为派生表。   
与子查询不同，派生表必须具有别名，以便稍后在查询中引用其名称。 如果派生表没有别名，MySQL将发出以下错误：   
Every derived table must have its own alias.

派生表的SQL语句如下：   
**SELECT column_list FROM(SELECT column_list FROM table_1) derived_table_name WHERE derived_table_name.c1 > 0;**

下面以orders和orderdetails两张表做示例，获得2013年销售收入最高的前5名产品：![ER-orders-orderdetails](./images/ER-orders-orderdetails.png)

先查询出的2013年的所有产品编号，金额。   
**SELECT productCode,(quantityOrdered * priceEach) AS total, YEAR(orderDate) AS year FROM orders INNER JOIN orderdetails USING(orderNumber) WHERE YEAR(orderDate)=2013;**

在2013年的结果中计算每个产品的销售额。   
**SELECT productCode,SUM(derived_table.total) AS avenue FROM (SELECT productCode,(quantityOrdered * priceEach) AS total, YEAR(orderDate) AS year FROM orders INNER JOIN orderdetails USING(orderNumber) WHERE YEAR(orderDate)=2013) derived_table GROUP BY productCode ORDER BY avenue DESC LIMIT 5;**

上面是使用到了派生表部分，当然也可以使用更简洁的SQL语句：   
**SELECT productCode,ROUND(SUM(quantityOrdered * priceEach)) AS sales FROM orderdetails INNER JOIN orders USING(orderNumber) WHERE YEAR(orderDate)=2013 GROUP BY productCode ORDER BY sales DESC LIMIT 5;**

也可以使用该查询结果作为派生表，并将其与products表相关联，products表的结构如下所示：![products-structure](./images/products-structure.png)

**SELECT productName,sales FROM products INNER JOIN (SELECT productCode,ROUND(SUM(quantityOrdered * priceEach)) AS sales FROM orderdetails INNER JOIN orders USING(orderNumber) WHERE YEAR(orderDate)=2013 GROUP BY productCode ORDER BY sales DESC LIMIT 5) top5products2013 USING(productCode);**

**较复杂的MySQL派生列表**

假设必须将2013年的客户分为3组：铂金，白金和白银。 此外，需要了解每个组中的客户数量，具体情况如下：
- 订单总额大于100000的为铂金客户；
- 订单总额为10000至100000的为黄金客户
- 订单总额为小于10000的为银牌客户

先查询出2013年每个客户的订单总额：   
**SELECT customerNumber,ROUND(SUM(quantityOrdered * priceEach)) AS sales FROM orders INNER JOIN orderdetails USING(orderNumber) WHERE YEAR(shippedDate) = 2013 GROUP BY customerNumber ORDER BY sales DESC;**

再使用CASE语句进行分组：   
**CASE 
WHEN SUM(quantityOrdered \* priceEach) < 10000 THEN 'Silver' 
WHEN SUM(quantityOrdered \* priceEach) BETWEEN 10000 AND 100000 THEN 'Gold' WHEN SUM(quantityOrdered * priceEach) > 100000 THEN 'Platinum' 
END**

组合起来得到每个客户属于哪个分组：   
**SELECT customerNumber,ROUND(SUM(quantityOrdered \* priceEach)) AS sales,(CASE WHEN SUM(quantityOrdered \* priceEach) < 10000 THEN 'Silver' WHEN SUM(quantityOrdered \* priceEach) BETWEEN 10000 AND 100000 THEN 'Gold' WHEN SUM(quantityOrdered \* priceEach) > 100000 THEN 'Platinum' END) customerGroup FROM orderdetails INNER JOIN orders USING(orderNumber) WHERE YEAR(shippedDate) = 2013 GROUP BY customerNumber ORDER BY sales DESC;**

再使用派生表，求出每个分组中客户的数量：    
**SELECT customerGroup,COUNT(customerGroup) AS groupCount FROM (SELECT customerNumber,ROUND(SUM(quantityOrdered \* priceEach)) AS sales,(CASE WHEN SUM(quantityOrdered \* priceEach) < 10000 THEN 'Silver' WHEN SUM(quantityOrdered \* priceEach) BETWEEN 10000 AND 100000 THEN 'Gold' WHEN SUM(quantityOrdered \* priceEach) > 100000 THEN 'Platinum' END) customerGroup FROM orderdetails INNER JOIN orders USING(orderNumber) WHERE YEAR(shippedDate) = 2013 GROUP BY customerNumber ORDER BY sales DESC) cg GROUP BY customerGroup;**